In [1]:
import requests
import collections
import time

import sqlite3 as lite

from dateutil.parser import parse
from IPython.core.display import clear_output

In [2]:
#Connect to database
con = lite.connect('citibike.db')
cur = con.cursor()

#Create bike data dictionary
bikeid=collections.defaultdict(int)

In [ ]:
for itr in range(0,60):
    #Pull JSON from CitiBike
    r = requests.get('http://www.citibikenyc.com/stations/json')
    
    print('Iteration: %d' % itr)
    #Pull execution time
    exec_time = parse(r.json()['executionTime'])

    #Create dictionary for available bikes per station
    for station in r.json()['stationBeanList']:
        bikeid[station['id']] = station['availableBikes']

    with con:
        #Insert execution time into database
        cur.execute('INSERT INTO available_bikes (execution_time) VALUES (?)', 
                    (exec_time.strftime("%Y%m%d_%H%M%S"),))

        #Insert available bikes per station into database
        for key,value in bikeid.items():
            cur.execute('UPDATE available_bikes SET _' 
                        + str(key) 
                        + ' = ' 
                        + str(value) 
                        + " WHERE execution_time = " 
                        + "'" 
                        + exec_time.strftime("%Y%m%d_%H%M%S")
                        + "'"
                        + ";")
    
    #Pause for 60 seconds till next iteration
    time.sleep(60)
    clear_output()